## Изменение формата

Так как исходные схемы сложны для понимания модели, они были упрощены. Упрощение производилось с помощью Autodesk AutoCAD 2022. Там можно сохранить сехмы в форматах png И jpeg, но в таком случае мы теряем сильно в качестве и теряем некоторые важные полоски и другое.

Следовательно требуется отдельный код, для того чтобы поменять формат и при этом не потерять качество изображения.

In [1]:
!pip install pdf2image

In [3]:
import os
from pdf2image import convert_from_path

# Путь к папке с poppler
poppler_path = 'poppler-24.08.0/Library/bin'

# Путь к папке с PDF-файлами
input_folder = 'examples_of_source_images'
# Путь к папке для сохранения PNG-файлов
output_folder = 'examples_of_source_images'

# Проверка, существует ли выходная папка, если нет — создаем её
os.makedirs(output_folder, exist_ok=True)

# Проход по всем файлам в папке
for filename in os.listdir(input_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(input_folder, filename)
        
        # Конвертация страниц PDF в изображения
        pages = convert_from_path(pdf_path, dpi=300, poppler_path=poppler_path)
        
        # Имя выходного файла, заменяем расширение на .png
        output_filename = os.path.splitext(filename)[0] + '.png'
        output_path = os.path.join(output_folder, output_filename)
        
        # Сохранение первой страницы как изображения в формате PNG
        pages[0].save(output_path, 'PNG')
        
        print(f"Сохранен: {output_path}")

Сохранен: examples_of_source_images\3д.png
Сохранен: examples_of_source_images\Благоустройство.png
Сохранен: examples_of_source_images\Вертикалка.png
Сохранен: examples_of_source_images\Картограмма.png
Сохранен: examples_of_source_images\Разбивка.png
Сохранен: examples_of_source_images\рельеф.png
Сохранен: examples_of_source_images\Сводный.png


## Информация о данных

### Все функции для анализа

In [1]:
import re
from collections import defaultdict
import os

In [2]:
# Регулярное выражение для поиска объектов и их количества, включая параметры
quantity_search_pattern = re.compile(r"([\w\s/=\d]+)\s(\d+)\s+\b(things|tracks|pier)\b")

# Регулярное выражение для поиска объектов (без учета количества)
pattern = re.compile(r"([\w\s/=\d]+)\s\d+\s+\b(things|tracks|pier)\b")


#### Функция для просмотра общего количества объектов на схемах

In [3]:
def total_number_of_objects(folder_path, pattern, object_counts):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    content = f.read()
                    # Ищем все совпадения с паттерном
                    matches = pattern.findall(content)
                    for obj, count, _ in matches:
                        # Суммируем количество встреченных объектов
                        object_counts[obj.strip()] += int(count)
    return object_counts

#### Функция для просмотра какие объекты в каждой схеме без учёта количества этих объектов

In [4]:
# Функция для обработки файлов
def objects_to_files(folder_path, pattern):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    content = f.read()
                    # Найти уникальные объекты в текущем файле
                    matches = set(pattern.findall(content))
                    print(f"В файле '{file}' найдены следующие объекты:")
                    for obj, _  in matches:
                        print(f" - {obj.strip()}")
                print()  # Пустая строка для разделения файлов

#### Функция для вывода инфомации о том в каком количестве файлов встречается тот или иной объект.

In [5]:
# Функция для обработки файлов
def files_with_objects(folder_path, pattern, files_counts):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    content = f.read()
                    # Найти уникальные объекты в текущем файле
                    matches = set(pattern.findall(content))
                    # Увеличить счетчик для каждого объекта
                    for obj, _ in matches:
                        files_counts[obj.strip()] += 1
    return files_counts

### Папка готовые данные

In [6]:
# Путь к папке с файлами
folder_path = "готовые данные/labels"

# Словарь для подсчета объектов
object_counts = defaultdict(int)

# Словарь для подсчёта файлов
files_counts = defaultdict(int)

#### Общее количество объектов по всем схемам

In [7]:
# Обработка файлов и вывод результата
object_counts = total_number_of_objects(folder_path=folder_path, pattern=quantity_search_pattern, object_counts=object_counts)
for obj, count in object_counts.items():
    print(f"{obj}: {count} times")


buildings administration/office: 24 times
production: 36 times
VST V=10000m3: 36 times
VST V=1000m3: 62 times
VST V=500m3: 56 times
VST V=100m3: 42 times
VST V=4500m3: 12 times
VST V=5000m3: 10 times
VST V=50m3: 54 times
VST V=3000m3: 8 times
HST V=75m3: 26 times
HST V=60m3: 12 times
HST V=50m3: 24 times
VST V=2000m3: 12 times


#### Информация о том, какие объекты в какой схеме

In [8]:
# Запуск функции обработки
objects_to_files(folder_path=folder_path, pattern=pattern)

В файле 'Елец_цвет.txt' найдены следующие объекты:
 - production
 - buildings administration/office
 - VST V=1000m3
 - VST V=100m3
 - VST V=500m3
 - VST V=10000m3

В файле 'Елец_чб.txt' найдены следующие объекты:
 - production
 - buildings administration/office
 - VST V=1000m3
 - VST V=100m3
 - VST V=500m3
 - VST V=10000m3

В файле 'Калининград_цвет.txt' найдены следующие объекты:
 - VST V=4500m3
 - production
 - buildings administration/office
 - VST V=1000m3
 - VST V=100m3
 - VST V=500m3

В файле 'Калининград_чб.txt' найдены следующие объекты:
 - VST V=4500m3
 - production
 - buildings administration/office
 - VST V=1000m3
 - VST V=100m3
 - VST V=500m3

В файле 'Кнайпеда_цвет.txt' найдены следующие объекты:
 - VST V=50m3
 - production
 - VST V=5000m3
 - buildings administration/office
 - VST V=1000m3
 - VST V=500m3
 - VST V=10000m3

В файле 'Кнайпеда_чб.txt' найдены следующие объекты:
 - VST V=50m3
 - production
 - VST V=5000m3
 - buildings administration/office
 - VST V=1000m3
 - VS

#### Информация о том, в каком количестве файлов встречается тот или иной объект

In [9]:
# Запуск функции обработки
files_with_objects(folder_path=folder_path, pattern=pattern, files_counts=files_counts)

# Вывод итогового результата
print("Объекты и количество их встреч во всех файлах:")
for obj, count in files_counts.items():
    print(f"{obj}: {count} files")


# Сортировка объектов по убыванию количества встреч
sorted_objects = sorted(files_counts.items(), key=lambda x: x[1], reverse=True)

# Вывод итогового результата
print("\n \n Объекты и количество их встреч во всех файлах (от самых частых):")
for obj, count in sorted_objects:
    print(f"{obj}: {count} files")

Объекты и количество их встреч во всех файлах:
production: 12 files
buildings administration/office: 14 files
VST V=1000m3: 16 files
VST V=100m3: 6 files
VST V=500m3: 14 files
VST V=10000m3: 10 files
VST V=4500m3: 4 files
VST V=50m3: 8 files
VST V=5000m3: 6 files
HST V=60m3: 2 files
HST V=75m3: 4 files
VST V=3000m3: 2 files
HST V=50m3: 4 files
VST V=2000m3: 2 files

 
 Объекты и количество их встреч во всех файлах (от самых частых):
VST V=1000m3: 16 files
buildings administration/office: 14 files
VST V=500m3: 14 files
production: 12 files
VST V=10000m3: 10 files
VST V=50m3: 8 files
VST V=100m3: 6 files
VST V=5000m3: 6 files
VST V=4500m3: 4 files
HST V=75m3: 4 files
HST V=50m3: 4 files
HST V=60m3: 2 files
VST V=3000m3: 2 files
VST V=2000m3: 2 files


#### Информация по схемам в виде датафрейма

In [10]:
import pandas

df = pandas.read_excel('готовые данные/data.xlsx')
df.head()

,image_path,image,buildings administration/office,production,Vertical steel tanker = 10000m3,Vertical steel tanker = 5000 m3,Vertical steel tanker = 4500 м3,Vertical steel tanker = 3000 м3,Vertical steel tanker = 2000 м3,Vertical steel tanker = 1000 м3,Vertical steel tanker = 500 м3,Vertical steel tanker = 100 м3,Vertical steel tanker = 50 м3,Horizontal steel tanker = 75 м3,Horizontal steel tanker = 60 м3,Horizontal steel tanker = 50 м3
0,data/готовые данные/images/Елец_цвет.png,NaN,2,4,4,0,0,0,0,3,4,8,0,0,0,0
1,data/готовые данные/images/Елец_чб.png,NaN,2,4,4,0,0,0,0,3,4,8,0,0,0,0
2,data/готовые данные/images/Калиниград_чб.png,NaN,1,4,0,0,4,0,0,2,2,5,0,0,0,0
3,data/готовые данные/images/Калининград_цвет.png,NaN,1,4,0,0,4,0,0,2,2,5,0,0,0,0
4,data/готовые данные/images/Кнайпеда_цвет.png,NaN,1,2,2,2,0,0,0,4,4,0,10,0,0,0


### Папка new_data

In [11]:
# Путь к папке с файлами
folder_path_new_data = "new_data/labels"

# Словарь для подсчета объектов
object_counts_new_data = defaultdict(int)

# Словарь для подсчёта файлов
files_counts_new_data = defaultdict(int)

#### Общее количество объектов по всем схемам

In [12]:
# Обработка файлов и вывод результата
object_counts_new_data = total_number_of_objects(folder_path_new_data, quantity_search_pattern, object_counts_new_data)
for obj, count in object_counts_new_data.items():
    print(f"{obj}: {count} times")

buildings administration/office: 12 times
production: 18 times
VST V=10000m3: 18 times
VST V=1000m3: 31 times
VST V=500m3: 28 times
VST V=100m3: 21 times
railway: 13 times
VST V=4500m3: 6 times
port: 1 times
VST V=5000m3: 5 times
VST V=50m3: 27 times
VST V=3000m3: 4 times
HST V=75m3: 13 times
HST V=60m3: 6 times
HST V=50m3: 12 times
VST V=2000m3: 6 times


#### Информация о том, какие объекты в какой схеме

In [13]:
# Запуск функции обработки
objects_to_files(folder_path_new_data, pattern)

В файле 'Елец_цвет.txt' найдены следующие объекты:
 - railway
 - production
 - buildings administration/office
 - VST V=1000m3
 - VST V=100m3
 - VST V=500m3
 - VST V=10000m3

В файле 'Калининград_цвет.txt' найдены следующие объекты:
 - railway
 - VST V=4500m3
 - production
 - buildings administration/office
 - VST V=1000m3
 - port
 - VST V=100m3
 - VST V=500m3

В файле 'Кнайпеда_цвет.txt' найдены следующие объекты:
 - VST V=50m3
 - railway
 - production
 - VST V=5000m3
 - buildings administration/office
 - VST V=1000m3
 - VST V=500m3
 - VST V=10000m3

В файле 'Колпино_1_цвет.txt' найдены следующие объекты:
 - VST V=50m3
 - railway
 - production
 - VST V=5000m3
 - buildings administration/office
 - VST V=1000m3
 - VST V=500m3
 - VST V=10000m3

В файле 'Колпино_2_цвет.txt' найдены следующие объекты:
 - VST V=50m3
 - railway
 - production
 - VST V=5000m3
 - VST V=1000m3
 - VST V=500m3
 - VST V=10000m3

В файле 'Сальск_цвет.txt' найдены следующие объекты:
 - HST V=60m3
 - VST V=50m3
 - HST

#### Информация о том, в каком количестве файлов встречается тот или иной объект

In [14]:
# Запуск функции обработки
files_with_objects(folder_path_new_data, pattern=pattern, files_counts=files_counts_new_data)

# Вывод итогового результата
print("Объекты и количество их встреч во всех файлах:")
for obj, count in files_counts_new_data.items():
    print(f"{obj}: {count} files")


# Сортировка объектов по убыванию количества встреч
sorted_objects = sorted(files_counts_new_data.items(), key=lambda x: x[1], reverse=True)

# Вывод итогового результата
print("\n \n Объекты и количество их встреч во всех файлах (от самых частых):")
for obj, count in sorted_objects:
    print(f"{obj}: {count} files")

Объекты и количество их встреч во всех файлах:
railway: 8 files
production: 6 files
buildings administration/office: 7 files
VST V=1000m3: 8 files
VST V=100m3: 3 files
VST V=500m3: 7 files
VST V=10000m3: 5 files
VST V=4500m3: 2 files
port: 1 files
VST V=50m3: 4 files
VST V=5000m3: 3 files
HST V=60m3: 1 files
HST V=75m3: 2 files
VST V=3000m3: 1 files
HST V=50m3: 2 files
VST V=2000m3: 1 files

 
 Объекты и количество их встреч во всех файлах (от самых частых):
railway: 8 files
VST V=1000m3: 8 files
buildings administration/office: 7 files
VST V=500m3: 7 files
production: 6 files
VST V=10000m3: 5 files
VST V=50m3: 4 files
VST V=100m3: 3 files
VST V=5000m3: 3 files
VST V=4500m3: 2 files
HST V=75m3: 2 files
HST V=50m3: 2 files
port: 1 files
HST V=60m3: 1 files
VST V=3000m3: 1 files
VST V=2000m3: 1 files


In [15]:
import pandas

df = pandas.read_excel('new_data/bitumen_terminal_new_data.xlsx')
df.head()

,image_path,image,buildings administration/office,production,Vertical steel tanker = 10000m3,Vertical steel tanker = 5000 m3,Vertical steel tanker = 4500 м3,Vertical steel tanker = 3000 м3,Vertical steel tanker = 2000 м3,Vertical steel tanker = 1000 м3,Vertical steel tanker = 500 м3,Vertical steel tanker = 100 м3,Vertical steel tanker = 50 м3,Horizontal steel tanker = 75 м3,Horizontal steel tanker = 60 м3,Horizontal steel tanker = 50 м3,railway,port
0,data/готовые данные/images/Елец_цвет.png,NaN,2,4,4,0,0,0,0,3,4,8,0,0,0,0,2,0
1,data/готовые данные/images/Калининград_цвет.png,NaN,1,4,0,0,4,0,0,2,2,5,0,0,0,0,1,1
2,data/готовые данные/images/Кнайпеда_цвет.png,NaN,1,2,2,2,0,0,0,4,4,0,10,0,0,0,2,0
3,data/готовые данные/images/Колпино_1_цвет.png,NaN,1,2,4,2,0,0,0,4,4,0,5,0,0,0,1,0
4,data/готовые данные/images/Колпино_2_цвет.png,NaN,0,3,4,1,0,0,0,2,6,0,10,0,0,0,1,0


Проверяем точно ли после конвертации таблица читается

In [1]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files='new_data/bitumen_terminal_new_data.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
dataset['train']['image_path']

['data/готовые данные/images/Елец_цвет.png',
 'data/готовые данные/images/Калининград_цвет.png',
 'data/готовые данные/images/Кнайпеда_цвет.png',
 'data/готовые данные/images/Колпино_1_цвет.png',
 'data/готовые данные/images/Колпино_2_цвет.png',
 'data/готовые данные/images/Сальск_цвет.png',
 'data/готовые данные/images/Талдом_цвет.png',
 'data/готовые данные/images/Тюмень_цвет.png',
 'data/готовые данные/images/Яссы_цвет.png']